# 内存管理

内存管理是由解释器自动处理的，开发者无需显式地进行内存分配和释放。Python 的内存管理机制主要包括以下几个方面：


## 对象缓存优化

针对特定不可变类型对象，在内存中缓存并重用。避免频繁的创建和销毁对象，减少内存分配的开销。


**小整数缓存**

小整数（`[-5, 256]`）。


In [125]:
x = 256
y = 256
x is y

True

**字符串驻留**

部分 `ASCII` 字符组成的字符串。


In [126]:
a = "hello_world"
b = "hello_world"
a is b

True

> `sys.intern` 手动创建驻留字符串。


In [127]:
import sys

a = sys.intern("你好")
b = sys.intern("你好")
a is b

True

**空元组缓存**


In [128]:
a = ()
b = ()
a is b

True

## 垃圾回收


**引用计数**

每当对象被引用时，引用计数加 1；当引用被删除或超出作用域时，引用计数减 1。当引用计数变为 0 时，对象被认为是不再被使用，将被垃圾回收。


In [129]:
from random import random
import sys

x = random()

sys.getrefcount(x)

2

In [130]:
sys.getsizeof(x)

24

In [131]:
ref = x

sys.getrefcount(x)

3

In [132]:
del ref

sys.getrefcount(x)

2

**标记-清除**

- 标记阶段：从根对象开始递归遍历对象图，标记可达对象。
- 清除阶段：标记完成后，垃圾回收器遍历整个内存，找到未标记对象，并释放其内存资源。

> `可达` 即仍被程序中其他对象所引用。


### 循环引用处理

两个或多个对象之间相互引用形成一个环即为循环引用，引用计数机制无法处理这种情况。


**循环垃圾收集器**

垃圾回收机制中的循环垃圾收集器会定期扫描内存中的对象，当循环引用的对象无法通过引用计数访问到时，垃圾回收机制会将它们标记为待回收对象。
垃圾回收机制会周期性地触发标记-清除算法，回收被标记待回收对象的内存。


In [133]:
import gc


class Node:
    def __init__(self, data):
        self.data = data
        self.next = None


# 创建循环引用
node1 = Node(1)
node2 = Node(2)
node1.next = node2
node2.next = node1

# 显式执行一次垃圾回收
# gc.collect()

# 检查循环引用是否被清除
print(gc.garbage)  # 输出结果：[]，表示循环引用已被成功清除

[]


**分代回收策略**


分代回收策略（Generational Collection Strategy）是一种常用的垃圾回收策略，用于管理和回收内存中的对象。该策略基于一个观察：大多数对象的生命周期较短，而只有少数对象具有较长的生命周期。因此，分代回收策略将对象根据其存活时间分为不同的代（Generation），并针对不同代采取不同的垃圾回收策略。

通常，分代回收策略将对象划分为三代：

1. 第 0 代（Young Generation）：第 0 代包含了新创建的对象。这些对象的生命周期较短，大部分对象很快就会变成垃圾。垃圾回收机制会频繁地对第 0 代进行回收，使用较轻量的垃圾回收算法，例如基于引用计数的垃圾回收。

2. 第 1 代（Intermediate Generation）：当对象在第 0 代经历了一定数量的垃圾回收后，仍然存活下来的对象会被晋升到第 1 代。第 1 代中的对象相对于第 0 代来说，具有更长的生命周期。垃圾回收机制会相对较少地对第 1 代进行回收，使用更耗时的垃圾回收算法，例如标记-清除算法。

3. 第 2 代（Old Generation）：第 2 代是存活时间最久的对象所在的代。这些对象的生命周期很长，只有很少一部分对象能够存活到这一代。垃圾回收机制会非常少地对第 2 代进行回收，使用更耗时的垃圾回收算法，例如标记-清除算法。

分代回收策略的核心思想是：根据对象的存活时间将其分配到不同的代中，并根据代的特点采用不同的垃圾回收策略。这样可以更加高效地管理内存，因为大多数对象都在年轻代中被回收，而只有少数对象需要经历更耗时的回收操作。
